## 画台风移动路径图
### 1. 库的导入及一些基本设置

In [ ]:
import sys
sys.path.append("/autodl-fs/data/examples")
import xarray as xr
from maptool import MapManager, np, plt, SystemRecognition
import pandas as pd

In [ ]:
model_name = "pangu" # 使用的模型
filepath = "/autodl-fs/data" # ERA5数据文件的路径
savepath = f"/root/autodl-tmp/result_{model_name}" # 结果保存路径
filename = f"{model_name}2022090718_lead32.nc" # 要读取的预测数据文件
target_time_period = slice("20220908 06:00", "20220915 00:00") #要画图的时刻

### 2. 分别读取ERA5数据和预测数据并检测路径

In [ ]:
ds = xr.open_dataset(f"{filepath}/9_surface.nc").sortby("latitude")
ds = ds.sel(longitude=slice(90, 140), latitude=slice(10, 40), time=target_time_period)

# SystemRecognition为检测系统中心的函数，通过检测局地极值中心来识别台风中心
# 但由于存在其它地区的低压干扰，往往容易偏，目前还没有很好的解决办法
# 使用此方法仅供演示使用，如有错误，请自行忽略
sr = SystemRecognition()
tmp = ds.msl.loc[..., 120:135]
p1 = sr.recognize_path(tmp.data, radius=2, center_status='low_equal', lat=tmp.latitude.data, lon=tmp.longitude.data, strategy='min')

ds = xr.open_dataset(f"{savepath}/{filename}").sortby("lat")
ds = ds.sel(lon=slice(90, 140), lat=slice(10, 40), time=target_time_period)
sr = SystemRecognition()
tmp = ds.msl.loc[..., 120:135]
p2 = sr.recognize_path(tmp.data, radius=8, center_status='low_equal', lat=tmp.lat.data, lon=tmp.lon.data, strategy='min')

### 3. 画两者的路径图

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
m = MapManager(ax)
m.extent([90, 140, 10, 40])
m.add_provinces(lw=0.9, color='gray')
ax.plot(*p1[0][::-1], 'r*-', label="ERA5")
ax.plot(*p2[0][::-1], 'b*-', label=model_name)
ax.legend()